In [120]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
import geopandas as gpd
from io import StringIO

In [121]:
with open('../data/api_toxen.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['api_key']

In [122]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.geojson'

In [123]:
params = {'offense_description':'BURGLARY- AGGRAVATED',  '$where': 'incident_reported between \'2021-01-01\' and \'2021-06-30\'','$limit': 10000}

In [124]:
response = requests.get(endpoint, params)
response

<Response [200]>

In [125]:
BA_gdf = gpd.read_file(StringIO(response.text))
BA_gdf.columns

Index(['victim_county_resident', 'zip_code', 'victim_number', 'offense_nibrs',
       'rpa', 'latitude', 'victim_race', 'incident_number',
       'investigation_status', 'offense_number', 'offense_description',
       'longitude', 'domestic_related', 'victim_ethnicity',
       'incident_location', 'victim_description', 'location_code',
       'report_type', 'incident_reported', 'weapon_description',
       'location_description', 'weapon_primary', 'report_type_description',
       'victim_type', 'incident_status_code', 'incident_status_description',
       'zone', 'victim_gender', 'incident_occurred', 'primary_key',
       'geometry'],
      dtype='object')

# 2

In [172]:
ctract_gdf = gpd.read_file("../data/tl_2019_47_tract.shp")
ctract_gdf = ctract_gdf.to_crs(epsg=4326)
ctract_gdf

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,47,165,021002,47165021002,210.02,Census Tract 210.02,G5020,S,30654374,207994,+36.3521602,-086.6464309,"POLYGON ((-86.69927 36.32775, -86.69923 36.327..."
1,47,165,020901,47165020901,209.01,Census Tract 209.01,G5020,S,23811766,3412697,+36.3483143,-086.4070907,"POLYGON ((-86.43949 36.35962, -86.43916 36.359..."
2,47,165,020300,47165020300,203,Census Tract 203,G5020,S,14629292,0,+36.5796910,-086.5114955,"POLYGON ((-86.53716 36.56851, -86.53713 36.568..."
3,47,165,020902,47165020902,209.02,Census Tract 209.02,G5020,S,9756715,0,+36.3724350,-086.4411274,"POLYGON ((-86.47106 36.38238, -86.47081 36.382..."
4,47,165,020800,47165020800,208,Census Tract 208,G5020,S,10235406,37743,+36.4005117,-086.4528100,"POLYGON ((-86.48786 36.37320, -86.48783 36.373..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,47,157,022112,47157022112,221.12,Census Tract 221.12,G5020,S,3293232,0,+35.0282414,-090.0009887,"POLYGON ((-90.01257 35.02328, -90.01256 35.023..."
1493,47,073,050501,47073050501,505.01,Census Tract 505.01,G5020,S,83253173,0,+36.5659218,-082.7714427,"POLYGON ((-82.85011 36.54106, -82.84955 36.541..."
1494,47,073,050601,47073050601,506.01,Census Tract 506.01,G5020,S,23852225,177932,+36.5836443,-082.6318663,"POLYGON ((-82.68939 36.58793, -82.68936 36.588..."
1495,47,073,050602,47073050602,506.02,Census Tract 506.02,G5020,S,23381992,668716,+36.5467218,-082.6458382,"POLYGON ((-82.69446 36.54855, -82.69445 36.548..."


In [131]:
BA_gdf = BA_gdf[
                ['victim_number',
                'incident_occurred', 
                'incident_number',
                'geometry']
                                    ]
                 

In [132]:
ctract_gdf = ctract_gdf[
                        ['NAMELSAD',
                        'geometry']
]

In [133]:
ab_by_censustract = gpd.sjoin(BA_gdf, ctract_gdf, op = 'within')
ab_by_censustract

,victim_number,incident_occurred,incident_number,geometry,index_right,NAMELSAD
0,1,2021-05-04T23:45:00,20210249540,POINT (-86.77000 36.15000),652,Census Tract 160
4,2,2021-05-04T23:45:00,20210249540,POINT (-86.77000 36.15000),652,Census Tract 160
63,1,2021-04-02T09:30:00,20210187070,POINT (-86.76000 36.14000),652,Census Tract 160
129,1,2021-01-04T10:45:00,20210006869,POINT (-86.76000 36.14000),652,Census Tract 160
158,1,2021-04-20T20:00:00,20210223220,POINT (-86.77000 36.15000),652,Census Tract 160
...,...,...,...,...,...,...
906,1,2021-06-24T14:00:00,20210344666,POINT (-86.60200 36.19800),567,Census Tract 154.05
925,2,2021-02-10T00:01:00,20210083306,POINT (-86.75800 36.05100),1067,Census Tract 188.01
1032,1,2021-02-10T00:01:00,20210083306,POINT (-86.75800 36.05100),1067,Census Tract 188.01
978,2,2021-04-22T09:45:00,20210225465,POINT (-86.82200 36.16800),98,Census Tract 136.02


In [134]:
ab_by_censustract_drop = ab_by_censustract.drop_duplicates(subset=['incident_number'])
ab_by_censustract_drop.groupby('NAMELSAD').count().nlargest(2,'victim_number')
                                                            

,victim_number,incident_occurred,incident_number,geometry,index_right
NAMELSAD,,,,,
Census Tract 163,53,53,53,53,53
Census Tract 162,21,21,21,21,21


Census Tract 163 had the highest number of aggrevated burglaries at 53.

In [171]:
ab_by_censustract

,victim_number,incident_occurred,incident_number,geometry,index_right,NAMELSAD
0,1,2021-05-04T23:45:00,20210249540,POINT (-86.77000 36.15000),652,Census Tract 160
4,2,2021-05-04T23:45:00,20210249540,POINT (-86.77000 36.15000),652,Census Tract 160
63,1,2021-04-02T09:30:00,20210187070,POINT (-86.76000 36.14000),652,Census Tract 160
129,1,2021-01-04T10:45:00,20210006869,POINT (-86.76000 36.14000),652,Census Tract 160
158,1,2021-04-20T20:00:00,20210223220,POINT (-86.77000 36.15000),652,Census Tract 160
...,...,...,...,...,...,...
906,1,2021-06-24T14:00:00,20210344666,POINT (-86.60200 36.19800),567,Census Tract 154.05
925,2,2021-02-10T00:01:00,20210083306,POINT (-86.75800 36.05100),1067,Census Tract 188.01
1032,1,2021-02-10T00:01:00,20210083306,POINT (-86.75800 36.05100),1067,Census Tract 188.01
978,2,2021-04-22T09:45:00,20210225465,POINT (-86.82200 36.16800),98,Census Tract 136.02


# 3

In [135]:
with open('../data/api_census_token.json') as fi:
    credentials = json.load(fi)
    
api_key_1 = credentials['api_key']

In [136]:
endpoint_1 = 'https://api.census.gov/data/2019/acs/acs5'

In [137]:
parameters_1 = {'get': 'B01001_001E', 'for': 'tract', 'in': 'state:47 county:037', 'key': api_key_1}

In [138]:
response_1 = requests.get(endpoint_1, parameters_1)
response_1

<Response [200]>

In [139]:
census_df= pd.DataFrame(response_1.json())
census_df.columns = census_df.iloc[0]
census_df = census_df.iloc[1:,:]
census_df.rename(columns = {'B01001_001E': 'population'}, inplace=True)
census_df

,population,state,county,tract
1,12176,47,037,015631
2,4098,47,037,015804
3,2466,47,037,017701
4,5210,47,037,017702
5,8254,47,037,018301
...,...,...,...,...
157,5252,47,037,015613
158,7262,47,037,015620
159,2345,47,037,016100
160,4617,47,037,011400


In [140]:
endpoint_2 = 'https://api.census.gov/data/2019/acs/acs5/subject'

In [141]:
parameters_2 = {'get': 'S1901_C01_012E', 'for': 'tract', 'in': 'state:47 county:037', 'key': api_key_1}

In [142]:
response_2 = requests.get(endpoint_2, parameters_2)
response_2

<Response [200]>

In [143]:
census_df_subject = pd.DataFrame(response_2.json())
census_df_subject.columns = census_df_subject.iloc[0]
census_df_subject = census_df_subject.iloc[1:,:]
census_df_subject.rename(columns= {'S1901_C01_012E': 'median_income'}, inplace=True)
census_df_subject

,median_income,state,county,tract
1,75579,47,037,015631
2,45129,47,037,015804
3,134786,47,037,017701
4,107813,47,037,017702
5,87591,47,037,018301
...,...,...,...,...
157,37656,47,037,015613
158,54149,47,037,015620
159,48221,47,037,016100
160,50857,47,037,011400


In [162]:
complete_census_df = pd.merge(census_df, census_df_subject, how='inner', on=['state', 'county', 'tract'])
complete_census_df = complete_census_df[['population', 'tract', 'median_income']]
complete_census_df 

,population,tract,median_income
0,12176,015631,75579
1,4098,015804,45129
2,2466,017701,134786
3,5210,017702,107813
4,8254,018301,87591
...,...,...,...
156,5252,015613,37656
157,7262,015620,54149
158,2345,016100,48221
159,4617,011400,50857


ValueError: 'left_df' should be GeoDataFrame, got <class 'pandas.core.frame.DataFrame'>